<a href="https://colab.research.google.com/github/SnoxP/Aumentar-FPS-em-video/blob/main/Aumentar_FPS_de_v%C3%ADdeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Veja aqui o tutorial [aqui](https://youtu.be/u_iLsWaHwyI) (espanhol)

# Bloco 1

In [ ]:
!git clone https://github.com/hzwer/arXiv2020-RIFE

# Bloco 2

In [ ]:
!mkdir -p /content/arXiv2020-RIFE/train_log
%cd /content/arXiv2020-RIFE/train_log
!gdown --id 1APIzVeI-4ZZCEuIRE1m6WYfSCaOsi_7_
!7z e RIFE_trained_model_v3.6.zip

# Bloco 3

Renomeie o nome do vídeo como "video.mp4" na pasta "arXiv2020-RIFE", execute o Bloco 3 e não reinicie a sessão caso o programa pedir

In [ ]:
%cd /content/arXiv2020-RIFE/

# Solução para o erro de interpolação (atualizado)
# Primeiro desinstalamos versões conflitantes
!pip uninstall -y numpy scikit-video

# Instalamos versões compatíveis
!pip install numpy==1.26.4
!pip install scikit-video==1.1.11

# Correção para numpy.float/np.float (atualizado para Python 3.11+)
!find /usr/local/lib/python3.*/dist-packages/skvideo/ -type f -name "*.py" -exec sed -i 's/np\.float/float/g; s/np\.int/int/g' {} +

# Baixar arquivo adicional
!gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc

# Bloco 4

In [ ]:
import os
import subprocess
from tqdm import tqdm
import re

%cd /content/arXiv2020-RIFE/

# Configuração
input_video = 'video.mp4'
output_audio = 'audio.aac'
interpolated_video = 'output.mp4'
final_video = 'video_final_interpolado.mp4'

# Verificar vídeo de entrada
if not os.path.exists(input_video):
    raise FileNotFoundError(f"Erro: O arquivo {input_video} não existe.")

# Extrair áudio (com manejo de erros melhorado)
audio_extracted = False
# Use subprocess to capture stderr for better error handling
try:
    result = subprocess.run(
        ['ffmpeg', '-y', '-i', input_video, '-vn', '-acodec', 'copy', output_audio],
        capture_output=True,
        text=True,
        check=True # Raise CalledProcessError if command returns non-zero exit code
    )
    if os.path.exists(output_audio) and os.path.getsize(output_audio) > 0:
        print("✅ Áudio extraído corretamente")
        audio_extracted = True
    else:
        # If ffmpeg succeeded but file is empty, it means no audio track
        if os.path.exists(output_audio):
            os.remove(output_audio)
        print("⚠️ O vídeo não contém áudio")
except subprocess.CalledProcessError as e:
    print(f"⚠️ Erro ao extrair áudio (pode ser que o vídeo não tenha áudio): {e.stderr.strip()}")
except FileNotFoundError:
     print("⚠️ Erro: ffmpeg não encontrado. Certifique-se de que está instalado.")


# Interpolar vídeo com barra de progresso
print("🔄 Interpolando vídeo...")

command = [
    'python3', 'inference_video.py',
    '--fps=60',
    '--exp=1',
    f'--video={input_video}',
    f'--output={interpolated_video}'
]

process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# Use tqdm para a barra de progresso
# A saída do script inference_video.py parece não ter um indicador de progresso numérico fácil.
# Vamos tentar capturar as linhas de saída como um indicador de progresso grosseiro,
# ou procurar por um padrão se houver.
# Se o script não fornecer um indicador numérico, a barra de progresso será baseada
# na quantidade de linhas de saída processadas.

# Estimativa do número total de "passos" ou linhas de saída esperadas
# Pode ser necessário ajustar este valor dependendo do vídeo e do script
estimated_total_steps = 100 # Valor inicial, pode ser ajustado após testes

with tqdm(total=estimated_total_steps, desc="Interpolando", unit="frame") as pbar:
    while True:
        output_line = process.stdout.readline()
        if output_line == '' and process.poll() is not None:
            break
        if output_line:
            # Tente encontrar um padrão numérico na saída que indique o progresso
            # Exemplo: procurando por números que possam representar o frame atual ou porcentagem
            match = re.search(r'(\d+)/\d+', output_line) # Exemplo: "Processing frame 10/100"
            if match:
                try:
                    current_frame = int(match.group(1))
                    # Se encontrarmos um indicador numérico, atualizamos a barra com ele
                    # Pode ser necessário um ajuste se o total não for detectado automaticamente
                    if pbar.total == estimated_total_steps: # Se ainda usando a estimativa
                         # Tente encontrar o total de frames na saída, se disponível
                         total_match = re.search(r'\d+/(\d+)', output_line)
                         if total_match:
                              pbar.total = int(total_match.group(1))
                              pbar.refresh() # Atualiza a barra com o novo total

                    pbar.n = current_frame
                    pbar.refresh()
                except ValueError:
                    # Se não conseguir converter para int, apenas ignore
                    pass
            else:
                 # Se nenhum padrão numérico for encontrado, apenas avance a barra por linha processada
                 # Isso dará uma indicação grosseira do progresso
                 pbar.update(1)
            # print(output_line.strip()) # Opcional: imprimir a saída do script

# Verificar se o subprocesso terminou com erro
if process.returncode != 0:
    raise RuntimeError(f"❌ Erro durante a interpolação do vídeo. Código de saída: {process.returncode}")

print("✅ Interpolação concluída")

# Combinar áudio e vídeo (se houver áudio)
if audio_extracted:
    print("🔊 Combinando áudio com vídeo interpolado...")
    try:
        subprocess.run(
            ['ffmpeg', '-y', '-i', interpolated_video, '-i', output_audio, '-c:v', 'copy', '-c:a', 'aac', '-strict', 'experimental', final_video],
            check=True,
            capture_output=True,
            text=True
        )
        print("✅ Áudio combinado corretamente")
        os.remove(interpolated_video)
    except subprocess.CalledProcessError as e:
        print(f"❌ Erro ao combinar áudio e vídeo: {e.stderr.strip()}")
        # Se a combinação falhar, manter o vídeo interpolado sem áudio
        os.rename(interpolated_video, final_video)
        print("🚫 Vídeo final gerado sem áudio devido a erro na combinação")
else:
    os.rename(interpolated_video, final_video)
    print("🚫 Vídeo final gerado sem áudio")

# Verificação final
if os.path.exists(final_video):
    print(f"🎉 Processo concluído! Vídeo final gerado: {final_video}")
    print(f"Tamanho do arquivo: {os.path.getsize(final_video)/1024/1024:.2f} MB")
else:
    raise RuntimeError("❌ Erro: Não foi possível gerar o vídeo final")